In [ ]:
# Install the latest versions of your required packages
!pip install numpy scipy h5py tensorflow keras
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import csv
import argparse
import matplotlib
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.models import load_model
import pandas as pd
from pathlib import Path
from keras.applications.xception import Xception, preprocess_input
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.losses import binary_crossentropy
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from datetime import datetime
import pytz
import time

In [ ]:
import urllib.request
# URLs for the zip files
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',
	  'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',
	  'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',
	  'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',
	  'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',
	  'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',
	  'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',
	  'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'
]


for idx, link in enumerate(links):
    fn = 'images_%02d.tar.gz' % (idx+1)
    print('downloading'+fn+'...')
    urllib.request.urlretrieve(link, fn)  # download the zip file

downloadingimages_01.tar.gz...
downloadingimages_02.tar.gz...
downloadingimages_03.tar.gz...
downloadingimages_04.tar.gz...
downloadingimages_05.tar.gz...
downloadingimages_06.tar.gz...
downloadingimages_07.tar.gz...
downloadingimages_08.tar.gz...
downloadingimages_09.tar.gz...
downloadingimages_10.tar.gz...
downloadingimages_11.tar.gz...
downloadingimages_12.tar.gz...


In [ ]:

! tar -xzvf images_01.tar.gz -C /content
! rm images_01.tar.gz
! tar -xzvf images_02.tar.gz -C /content
! rm images_02.tar.gz
! tar -xzvf images_03.tar.gz -C /content
! rm images_03.tar.gz
! tar -xzvf images_04.tar.gz -C /content
! rm images_04.tar.gz
! tar -xzvf images_05.tar.gz -C /content
! rm images_05.tar.gz
! tar -xzvf images_06.tar.gz -C /content
! rm images_06.tar.gz
! tar -xzvf images_07.tar.gz -C /content
! rm images_07.tar.gz
! tar -xzvf images_08.tar.gz -C /content
! rm images_08.tar.gz
! tar -xzvf images_09.tar.gz -C /content
! rm images_09.tar.gz
! tar -xzvf images_10.tar.gz -C /content
! rm images_10.tar.gz
! tar -xzvf images_11.tar.gz -C /content
! rm images_11.tar.gz
! tar -xzvf images_12.tar.gz -C /content
! rm images_12.tar.gz

串流輸出內容已截斷至最後 5000 行。
images/00030152_007.png
images/00029217_004.png
images/00030742_000.png
images/00030561_002.png
images/00030313_000.png
images/00029727_002.png
images/00030077_004.png
images/00028518_025.png
images/00028715_002.png
images/00030642_000.png
images/00028253_006.png
images/00028341_002.png
images/00030158_002.png
images/00029808_000.png
images/00029470_012.png
images/00028888_001.png
images/00029041_012.png
images/00029118_007.png
images/00028422_000.png
images/00030239_000.png
images/00029708_000.png
images/00029880_010.png
images/00028523_031.png
images/00028211_013.png
images/00028751_000.png
images/00028322_000.png
images/00030568_000.png
images/00030079_038.png
images/00030139_000.png
images/00028939_010.png
images/00030059_005.png
images/00028651_000.png
images/00029234_000.png
images/00029701_002.png
images/00030468_000.png
images/00030039_000.png
images/00028980_000.png
images/00029563_000.png
images/00030797_000.png
images/00029134_000.png
images/00028652_003

In [ ]:
os.chdir('/content/drive/MyDrive/Machine-learning-project/Xception')


In [ ]:
def get_timestamp():
    tz = pytz.timezone('Asia/Singapore')
    now = datetime.now(tz)
    return now.strftime("%m%d_%H%M")  # Format: MMDD_HHMM

def save_batch_to_csv(batch_logs, file_path, batch_idx):
    mode = 'a' if os.path.exists(file_path) else 'w'
    with open(file_path, mode, newline='') as csvfile:
        writer = csv.writer(csvfile)
        if mode == 'w': writer.writerow(['batch'] + list(batch_logs.keys()))
        row = [batch_idx] + [batch_logs[key] for key in batch_logs.keys()]
        writer.writerow(row)

class BatchMetricsCallback(Callback):
    def __init__(self, output_csv_path):
        #Args: output_csv_path: Path to the CSV file where batch metrics will be saved.
        super().__init__()
        self.output_csv_path = output_csv_path

    def on_batch_end(self, batch, logs=None): #This function is called at the end of each batch.
        #Logs metrics to a CSV file after each batch.
        # Start the timer before the batch processing
        start_time = time.time()
        # Extract relevant metrics (e.g., loss, accuracy, AUC)
        batch_loss = logs.get('loss')
        batch_accuracy = logs.get('accuracy')
        batch_auc = logs.get('auc')

        # Calculate the elapsed time for this batch
        elapsed_time = time.time() - start_time

        # Prepare the log data
        batch_logs = {
            'loss': batch_loss,'accuracy': batch_accuracy,
            'auc': batch_auc,'elapsed_time': elapsed_time  # Add elapsed time to the logs
        }
        # Log the batch metrics to CSV
        with open(self.output_csv_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            if csvfile.tell() == 0:  # If file is empty, write the header
                writer.writerow(['batch', 'loss', 'accuracy', 'auc', 'elapsed_time'])
            writer.writerow([batch] + [batch_logs[key] for key in batch_logs.keys()])

def save_val_results_to_csv(val_data, val_labels, model, val_csv_paths, epoch):
    predictions = model.predict(val_data)
    for val_csv_path in val_csv_paths:
        df = pd.DataFrame(predictions, columns=['Prediction'])
        df['True Labels'] = val_labels
        df['Epoch'] = epoch
        df.to_csv(val_csv_path, mode='a', header=not os.path.exists(val_csv_path), index=False)

class ValidationCallback(Callback):
    def __init__(self, val_datasets, val_csv_paths):
        #val_datasets (list): List of validation datasets.
        #val_csv_paths (list): List of file paths for saving validation metrics.
        super().__init__()
        self.val_datasets = val_datasets
        self.val_csv_paths = val_csv_paths

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        # Iterate through the validation datasets and output metrics
        for val_dataset, val_csv_path in zip(self.val_datasets, self.val_csv_paths):
            val_loss, val_accuracy, val_auc = 0, 0, 0
            count = 0
            # Validate on the entire validation dataset
            for val_data, val_labels in val_dataset:
                predictions = self.model.predict(val_data)

                # Convert labels to NumPy if needed
                val_labels_np = val_labels.numpy() if isinstance(val_labels, tf.Tensor) else val_labels
                predictions_np = predictions.numpy() if isinstance(predictions, tf.Tensor) else predictions

                # Calculate loss, accuracy, and AUC
                val_loss += tf.keras.losses.categorical_crossentropy(val_labels, predictions).numpy().mean()
                val_accuracy += tf.keras.metrics.categorical_accuracy(val_labels, predictions).numpy().mean()
                val_auc += roc_auc_score(val_labels_np.ravel(), predictions_np.ravel())
                count += 1

            # Compute average metrics
            metrics = {
                'val_loss': val_loss / count,
                'val_accuracy': val_accuracy / count,
                'val_auc': val_auc / count,
            }
            # Save metrics to CSV file for each validation dataset
            save_metrics_to_csv(metrics, val_csv_path, epoch + 1)


# Helper function to create datasets  # Data Generators
def create_datasets(train_input_paths, train_labels, val_input_paths, val_labels, batch_size):
    train_dataset = tf.data.Dataset.from_tensor_slices((train_input_paths, train_labels))
    train_dataset = (train_dataset
                     .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
                     .shuffle(buffer_size=100)
                     .batch(batch_size)
                     .prefetch(tf.data.AUTOTUNE))

    val_dataset = tf.data.Dataset.from_tensor_slices((val_input_paths, val_labels))
    val_dataset = (val_dataset
                   .map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
                   .batch(batch_size)
                   .prefetch(tf.data.AUTOTUNE))
    return train_dataset, val_dataset

def load_and_preprocess_image(path, label):
    # Load the image file
    image = tf.io.read_file(path)
    # Decode and resize the image
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [299, 299])  # Adjusted to Xception's input size
    # Preprocess for Xception
    image = tf.keras.applications.xception.preprocess_input(image)
    return image, label

class BatchLoggerCallback(Callback):
    def __init__(self, log_path, val_datasets, val_csv_paths):
        super().__init__()
        self.log_path = log_path
        self.val_datasets = val_datasets
        self.val_csv_paths = val_csv_paths

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        save_batch_to_csv(logs, self.log_path, batch)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for val_dataset, val_csv_path in zip(self.val_datasets, self.val_csv_paths):
            val_data, val_labels = next(iter(val_dataset))
            save_val_results_to_csv(val_data, val_labels, self.model, val_csv_path, epoch)

def generate_from_paths_and_labels(
        input_paths, labels, batch_size, input_size=(299, 299)):
    num_samples = len(input_paths)
    while 1:
        perm = np.random.permutation(num_samples)
        input_paths = input_paths[perm]
        labels = labels[perm]
        for i in range(0, num_samples, batch_size):
            inputs = list(map(
                lambda x: image.load_img(x, target_size=input_size),
                input_paths[i:i+batch_size]
            ))
            inputs = np.array(list(map(
                lambda x: image.img_to_array(x),
                inputs
            )))
            inputs = preprocess_input(inputs)
            yield (inputs, labels[i:i+batch_size])

def generate_from_paths_and_multi_labels(input_paths, labels, batch_size, input_size=(299, 299)):
    """
    Data generator function for multi-label classification.
    Args:
        input_paths (list): List of file paths for images.
        labels (array): Corresponding multi-hot encoded labels.
        batch_size (int): Number of images per batch.
        input_size (tuple): Target size for images, defaults to (299, 299).
    Yields:
        Tuple of (inputs, labels) where inputs is a batch of processed images
        and labels is a batch of multi-hot encoded labels.
    """
    num_samples = len(input_paths)
    while True:
        # Shuffle data at the beginning of each epoch
        perm = np.random.permutation(num_samples)
        input_paths, labels = input_paths[perm], labels[perm]

        for i in range(0, num_samples, batch_size):
            # Load and preprocess images in the current batch
            batch_inputs = [image.load_img(x, target_size=input_size) for x in input_paths[i:i+batch_size]]
            batch_inputs = np.array([image.img_to_array(img) for img in batch_inputs])
            batch_inputs = preprocess_input(batch_inputs)

            # Yield a tuple of inputs and corresponding labels
            yield batch_inputs, labels[i:i+batch_size]

def save_history_to_csv(history, file_path, append=False):
    """
    Save training history to a CSV file.
        history (dict): Training history with keys and lists of metrics values.
        file_path (str): Path to the CSV file.
        append (bool): If True, append to existing CSV; otherwise, create a new file.
    """
    mode = 'a' if append else 'w'
    with open(file_path, mode, newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write headers if not appending
        if not append:
            writer.writerow(['epoch'] + list(history.keys()))

        # Write metric values for each epoch
        num_epochs = len(list(history.values())[0])
        for epoch in range(num_epochs):
            row = [epoch + 1] + [history[key][epoch] for key in history.keys()]
            writer.writerow(row)

def save_metrics_to_csv(logs, file_path, epoch):
    """
    Save validation metrics to a CSV file.
        logs (dict): Dictionary containing metrics (e.g., loss, accuracy, AUC).
        file_path (str): Path to the output CSV file.
        epoch (int): Current epoch number.
    """
    mode = 'a' if os.path.exists(file_path) else 'w'
    with open(file_path, mode, newline='') as csvfile:
        writer = csv.writer(csvfile)
        # Write header if creating a new file
        if mode == 'w':
            writer.writerow(['epoch'] + list(logs.keys()))
        # Write metrics
        row = [epoch] + [logs[key] for key in logs.keys()]
        writer.writerow(row)

class LastEpochROCCallback(Callback):
    def __init__(self, val_datasets, val_labels, csv_paths, csv_paths_pre,class_names):
        super().__init__()
        self.val_datasets = val_datasets
        self.val_labels = val_labels
        self.csv_paths = csv_paths
        self.class_names = class_names
        self.csv_paths_pre=csv_paths_pre

    def on_epoch_end(self, epoch, logs=None):
        # Generate ROC curves only on the last epoch
        if epoch == self.params['epochs'] - 1:
            for i, (val_data, val_labels) in enumerate(zip(self.val_datasets, self.val_labels)):
                y_pred = self.model.predict(val_data)
                # Save to CSV
                csv_path_pre = self.csv_paths_pre[i]
                df = pd.DataFrame(y_pred)
                df.to_csv(csv_path_pre, index=False, mode='w' if epoch == 0 else 'a', header=not os.path.exists(csv_path_pre))

                # Dataframe to store FPR, TPR, AUC for each class
                df = pd.DataFrame()

                # Compute ROC curve and AUC for each class
                for class_index, class_name in enumerate(self.class_names):
                    fpr, tpr, _ = roc_curve(val_labels[:, class_index], y_pred[:, class_index])
                    auc_value = roc_auc_score(val_labels[:, class_index], y_pred[:, class_index])

                    # Save to CSV
                    csv_path = self.csv_paths[i]
                    df = pd.DataFrame({'FPR': fpr, 'TPR': tpr, 'AUC': [auc_value] * len(fpr),'Class': "Total"})
                    df.to_csv(csv_path, index=False, mode='w' if epoch == 0 else 'a', header=not os.path.exists(csv_path))

def compute_roc_and_save(predictions, true_labels, classes, output_csv_path):
    """
    Computes ROC metrics for each class and saves them to a CSV file.

    Args:
        predictions (ndarray): Model predictions of shape (n_samples, n_classes).
        true_labels (ndarray): Ground truth labels of shape (n_samples, n_classes).
        classes (list): List of class names.
        output_csv_path (str): Path to save the CSV file.
    """
    all_data = []

    for idx, class_name in enumerate(classes):
        fpr, tpr, _ = roc_curve(true_labels[:, idx], predictions[:, idx])
        auc_score = roc_auc_score(true_labels[:, idx], predictions[:, idx])

        # Combine metrics into a DataFrame
        class_data = pd.DataFrame({
            'FPR': fpr,
            'TPR': tpr,
            'AUC': [auc_score] * len(fpr),
            'Class': [class_name] * len(fpr)
        })
        all_data.append(class_data)

    # Concatenate all class data and save to CSV
    combined_data = pd.concat(all_data, axis=0)
    combined_data.to_csv(output_csv_path, index=False)
    print(f"ROC metrics saved to {output_csv_path}")

class ROCFinalEpochCallback(Callback):
    def __init__(self, val_datasets, val_labels, val_csv_paths, classes):
        """
        Args:
            val_datasets (list): List of validation datasets.
            val_labels (list): List of ground truth labels for each validation set.
            val_csv_paths (list): Paths to save the ROC CSVs for each validation set.
            classes (list): List of class names.
        """
        super().__init__()
        self.val_datasets = val_datasets
        self.val_labels = val_labels
        self.val_csv_paths = val_csv_paths
        self.classes = classes

    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 == self.params['epochs']:  # Check if it's the last epoch
            for val_data, val_labels, csv_path in zip(self.val_datasets, self.val_labels, self.val_csv_paths):
                predictions = self.model.predict(val_data)
                compute_roc_and_save(predictions, val_labels, self.classes, csv_path)


In [ ]:
matplotlib.use('Agg')
#sys.argv = ['colab_launcher.py', '/content/images', 'classes.txt', 'result/','Data_Entry_2017.csv']
#sys.argv = ['colab_launcher.py', '/content', 'classes.txt', 'result','/content/drive/MyDrive/train_data20241111/train_young.csv','/content/drive/MyDrive/train_data20241111/valid_young.csv']
#sys.argv = ['colab_launcher.py', '/content', 'classes.txt', 'result','/content/drive/MyDrive/train_data20241111/train_middle.csv','/content/drive/MyDrive/train_data20241111/valid_middle.csv']
#sys.argv = ['colab_launcher.py', '/content', 'classes.txt', 'result','/content/drive/MyDrive/train_data20241111/train_old.csv','/content/drive/MyDrive/train_data20241111/valid_old.csv']
#sys.argv = ['colab_launcher.py', '/content', 'classes.txt', 'result','/content/drive/MyDrive/train_data20241111/train_F.csv','/content/drive/MyDrive/train_data20241111/valid_F.csv']
sys.argv = ['colab_launcher.py', '/content', 'classes.txt', 'result','/content/drive/MyDrive/train_data20241111/train_old.csv','/content/drive/MyDrive/train_data20241111/valid_M.csv',
       '/content/drive/MyDrive/train_data20241111/valid_young.csv','/content/drive/MyDrive/train_data20241111/valid_middle.csv','/content/drive/MyDrive/train_data20241111/valid_old.csv','/content/drive/MyDrive/train_data20241111/valid_F.csv',
       '','','','','']

parser = argparse.ArgumentParser()
parser.add_argument('dataset_root')
parser.add_argument('classes')
parser.add_argument('result_root')
parser.add_argument('train_csv')  # Path to the training CSV file
parser.add_argument('val_csv_M')  # Path to the validation CSV file
parser.add_argument('val_csv_young')
parser.add_argument('val_csv_middle')
parser.add_argument('val_csv_old')
parser.add_argument('val_csv_F')
parser.add_argument('val_M_csv_paths')
parser.add_argument('val_young_csv_paths')
parser.add_argument('val_middle_csv_paths')
parser.add_argument('val_olf_csv_paths')
parser.add_argument('val_F_csv_paths')

#pre的部分，輸入上次是多少，改--epochs_pre到最終層數(eg.從第三層到第六層)
#fine的部分，輸入還要在幾層(eg.過程需要1層)
parser.add_argument('--epochs_pre', type=int, default=9)
parser.add_argument('--epochs_fine', type=int, default=1)
parser.add_argument('--batch_size_pre', type=int, default=64)
parser.add_argument('--batch_size_fine', type=int, default=32)
parser.add_argument('--lr_pre', type=float, default=1e-4)
parser.add_argument('--lr_fine', type=float, default=1e-5)
parser.add_argument('--snapshot_period_pre', type=int, default=1)
parser.add_argument('--snapshot_period_fine', type=int, default=1)
parser.add_argument('--load_model', type=str, default=False, help='Path to a saved model to continue training')
args = parser.parse_args()

last = 0 #pre上次是多少
#"result/1109_0033/model_pre_ep01_val_loss0.3195.keras" #替換'--load_model中的default=False 輸入所需路徑
#path_name = "result/1113_1400" # where your desired model path here
#os.makedirs(path_name, exist_ok=True)
#print(args.train_csv)

In [ ]:
def main(args):
    time=f'10_Epochs_old__{get_timestamp()}'
    args.result_root = os.path.join('result/', time)
    os.makedirs(args.result_root, exist_ok=True)
    train_df = pd.read_csv(args.train_csv)
    val_M_df = pd.read_csv(args.val_csv_M)
    val_young_df = pd.read_csv(args.val_csv_young)
    val_middle_df = pd.read_csv(args.val_csv_middle)
    val_old_df = pd.read_csv(args.val_csv_old)
    val_F_df = pd.read_csv(args.val_csv_F)
    val_dfs = [val_M_df,val_young_df,val_middle_df,val_old_df,val_F_df]

    train_input_paths = [os.path.join(args.dataset_root, fname) for fname in train_df['Path']]
    val_M_input_paths = [os.path.join(args.dataset_root, fname) for fname in val_M_df['Path']]
    # Paths for the validation datasets
    val_young_input_paths = [os.path.join(args.dataset_root, fname) for fname in val_young_df['Path']]
    val_middle_input_paths = [os.path.join(args.dataset_root, fname) for fname in val_middle_df['Path']]
    val_old_input_paths = [os.path.join(args.dataset_root, fname) for fname in val_old_df['Path']]
    val_F_input_paths = [os.path.join(args.dataset_root, fname) for fname in val_F_df['Path']]
    val_input_paths = [val_M_input_paths,val_young_input_paths,val_middle_input_paths,val_old_input_paths,val_F_input_paths]

  # Load and preprocess labels
    label_columns = [col for col in train_df.columns if col != 'Path']
    train_labels = train_df[label_columns].values
    val_M_labels = val_M_df[label_columns].values
    val_young_labels = val_young_df[label_columns].values
    val_middle_labels = val_middle_df[label_columns].values
    val_old_labels = val_old_df[label_columns].values
    val_F_labels = val_F_df[label_columns].values
    val_labels = [val_M_labels,val_young_labels,val_middle_labels,val_old_labels,val_F_labels]

    # Create validation datasets
    train_dataset,val_M_dataset = create_datasets(train_input_paths, train_labels, val_M_input_paths,val_M_labels,args.batch_size_pre)
    train_dataset1,val_young_dataset = create_datasets(train_input_paths, train_labels, val_young_input_paths, val_young_labels, args.batch_size_pre)
    train_dataset2,val_middle_dataset = create_datasets(train_input_paths, train_labels, val_middle_input_paths, val_middle_labels, args.batch_size_pre)
    train_dataset3,val_old_dataset = create_datasets(train_input_paths, train_labels, val_old_input_paths, val_old_labels, args.batch_size_pre)
    train_dataset4,val_F_dataset = create_datasets(train_input_paths, train_labels, val_F_input_paths, val_F_labels, args.batch_size_pre)
    val_datasets = [val_M_dataset,val_young_dataset,val_middle_dataset,val_old_dataset,val_F_dataset]

    # Paths for validation CSV outputs
    #args.result_root = os.path.join('result/', time)
    args.val_M_csv_paths = os.path.join(args.result_root,'val_M.csv')
    print(args.val_M_csv_paths)
    args.val_young_csv_paths = os.path.join(args.result_root, 'val_young.csv')
    args.val_middle_csv_paths = os.path.join(args.result_root, 'val_middle.csv')
    args.val_old_csv_paths = os.path.join(args.result_root, 'val_old.csv')
    args.val_F_csv_paths = os.path.join(args.result_root, 'val_F.csv')
    val_csv_paths = [args.val_M_csv_paths,args.val_young_csv_paths,args.val_middle_csv_paths,args.val_old_csv_paths,args.val_F_csv_paths]
    val_roc_csv_paths = [os.path.join(args.result_root,'roc_M.csv'),os.path.join(args.result_root,'roc_young.csv'),os.path.join(args.result_root,'roc_middle.csv'),
                         os.path.join(args.result_root,'roc_old.csv'),os.path.join(args.result_root,'roc_F.csv') ]
    val_roc_csv_paths_total = [os.path.join(args.result_root,'roc_M_total.csv'),os.path.join(args.result_root,'roc_young_total.csv'),os.path.join(args.result_root,'roc_middle_total.csv'),
                               os.path.join(args.result_root,'roc_old_total.csv'),os.path.join(args.result_root,'roc_F_total.csv') ]
    csv_paths_pre =[os.path.join(args.result_root,'roc_M_pre.csv'),os.path.join(args.result_root,'roc_young_pre.csv'),os.path.join(args.result_root,'roc_middle_pre.csv'),
                               os.path.join(args.result_root,'roc_old_pre.csv'),os.path.join(args.result_root,'roc_F_pre.csv') ]

    unique_classes = label_columns
    num_classes = len(unique_classes)
    with open(args.classes, 'w') as f:
        for class_name in unique_classes:
            f.write(f"{class_name}\n")
    print(f"classes.txt created successfully with the following class names:\n{unique_classes}")

    # Compute class weights for handling imbalanced classes
    labels = np.concatenate([train_labels, val_M_labels,val_young_labels,val_middle_labels,val_old_labels,val_F_labels], axis=0)
    class_weights = compute_class_weight(class_weight='balanced', classes=np.arange(len(unique_classes)), y=np.argmax(labels, axis=1))
    class_weights = {i: weight for i, weight in enumerate(class_weights)}

    # ====================================================
    # Build or Load Model
    print(args.load_model)
    if args.load_model:
        model = load_model(args.load_model, custom_objects={
            'AUC': AUC,
            'Precision': Precision,
            'Recall': Recall,
        })
        print(f'Model loaded from "{args.load_model}" for continued training.')
        last_fine = args.epochs_pre  # Fine-tuning will start from the same epoch unless pre-training was completed

    else:
        base_model = Xception(include_top=False, weights='imagenet', input_shape=(299, 299, 3))
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        predictions = Dense(num_classes, activation='sigmoid')(x)
        model = Model(inputs=base_model.inputs, outputs=predictions)
        for layer in base_model.layers:
            layer.trainable = False
        last_fine = args.epochs_pre  # Fine-tuning starts after pre-training is complete
        print(last_fine)

    # Compile Model
    learning_rate = args.lr_fine if args.load_model else args.lr_pre
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss=binary_crossentropy,
        metrics=['accuracy', AUC(name='auc'), Precision(name='precision'),Recall(name='recall')]
    )

    # Set output CSV path for batch-level metrics
    batch_metrics_csv_path = os.path.join(args.result_root, 'batch_metrics.csv')

    # Instantiate the BatchMetricsCallback
    batch_metrics_callback = BatchMetricsCallback(batch_metrics_csv_path)
    val_callback = ValidationCallback(val_datasets,val_csv_paths)
    roc_callback_total = LastEpochROCCallback(val_datasets, val_labels, val_roc_csv_paths ,csv_paths_pre, unique_classes)
    roc_callback = ROCFinalEpochCallback(val_datasets,val_labels, val_roc_csv_paths,unique_classes)


    # Training Phase
    if 1: #not args.load_model:
        hist_pre = model.fit(
            train_dataset,
            epochs=args.epochs_pre,
            initial_epoch = last,
            validation_data=val_M_dataset,
            class_weight=class_weights,
            verbose=1,
            callbacks=[val_callback,batch_metrics_callback, ModelCheckpoint(
                filepath=os.path.join(args.result_root, 'model_pre_ep{epoch:02d}_val_loss{val_loss:.4f}.keras'),
                save_freq='epoch',  monitor='val_loss', mode='min'
            )]
        )
        if (args.epochs_pre!=last):
            print(hist_pre.history.keys())
            pre_history_csv_path = os.path.join(args.result_root, 'training_history.csv')
            save_history_to_csv(hist_pre.history, pre_history_csv_path, append=bool(args.load_model))

        for layer in model.layers:
          layer.trainable = True

        model.compile(
          optimizer=Adam(learning_rate=learning_rate),
          loss=binary_crossentropy,
           metrics=['accuracy', AUC(name='auc'), Precision(name='precision'),Recall(name='recall')]
        )

    # Fine-Tuning Phase
    hist_fine = model.fit(
        train_dataset,
        epochs=args.epochs_fine+last_fine,
        initial_epoch = last_fine,
        validation_data=val_M_dataset,
        class_weight=class_weights,
        verbose=1,
        callbacks=[val_callback,batch_metrics_callback,roc_callback,roc_callback_total,ModelCheckpoint(
            filepath=os.path.join(args.result_root, 'model_fine_ep{epoch:02d}_val_loss{val_loss:.4f}.keras'),
            save_freq='epoch', monitor='val_loss', mode='min')]
    )

    print(hist_fine.history.keys())
    fine_history_csv_path = os.path.join(args.result_root, 'training_history.csv')
    if (args.epochs_fine+last_fine!=last_fine):
        save_history_to_csv(hist_fine.history, fine_history_csv_path, append=True)

if __name__ == '__main__':
    main(args)

result/10_Epochs_old__1123_0057/val_M.csv
classes.txt created successfully with the following class names:
['Nodule', 'Infiltration', 'Effusion', 'Atelectasis']
False
9
Epoch 1/9
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 238ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
2/

In [ ]:
##把Roc的每個類別分開
#有**符號的是要更改的部分

import pandas as pd

# Load your dataset (replace 'your_file.csv' with the actual file name)
#**roc路徑
file_path = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_old.csv'
df = pd.read_csv(file_path)

# Create a separate DataFrame for each class
classes = df['Class'].unique()
class_dataframes = {
    cls: df[df['Class'] == cls].drop(columns=['Class']).reset_index(drop=True).rename(
        columns=lambda x: f"{cls}_{x}") for cls in classes
}

# Add a blank column for separation
spacer = pd.DataFrame({" ": [""] * len(next(iter(class_dataframes.values())))})  # Blank column with the same number of rows

# Combine the DataFrames side-by-side with blank columns in between
dataframes_with_spacers = []
for cls in classes:
    dataframes_with_spacers.append(class_dataframes[cls])
    dataframes_with_spacers.append(spacer)

# Combine everything horizontally
combined_df = pd.concat(dataframes_with_spacers, axis=1).iloc[:, :-1]  # Remove the last blank column

# Save the result to a new CSV file
#**存取路徑
combined_df.to_csv("/content/drive/MyDrive/Machine-learning-project/Xception/1123_result/F/F_roc_old_separated_classes.csv", index=False)

print("Classes have been separated with spacing columns and saved to 'separated_classes_with_spacing.csv'.")


OSError: Cannot save file into a non-existent directory: '/content/drive/MyDrive/Machine-learning-project/Xception/1123_result/F'

In [ ]:
##把預測的資料集中在一資料當中
#有**符號的是要更改的部分

#**檔案路徑
# File paths for the 4 input files
M_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_M_pre.csv'
F_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_F_pre.csv'
young_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_young_pre.csv'
middle_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_middle_pre.csv'
old_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/roc_old_pre.csv'

# File paths for the 4 input files
file_paths = [M_csv, F_csv, young_csv, middle_csv,old_csv]

# Group names corresponding to each file
group_names = ['M', 'F', 'young', 'middle','old']

# List to hold DataFrames
dataframes = []

# Process each file and format it
for file_path, group_name in zip(file_paths, group_names):
    # Read the CSV file
    df = pd.read_csv(file_path, header=None)

    # Add the "Index" column specific to the group
    df.insert(0, 'Index', range(1, len(df) + 1))

    # Rename the remaining columns dynamically based on the group name
    df.columns = ['Index',
                  f'{group_name}_Nodule',
                  f'{group_name}_Infiltration',
                  f'{group_name}_Effusion',
                  f'{group_name}_Atelectasis']

    # Append the group DataFrame to the list
    dataframes.append(df)

# Align all DataFrames to have the same number of rows by padding with NaN
max_rows = max(len(df) for df in dataframes)
aligned_dataframes = [df.reindex(range(max_rows)) for df in dataframes]

# Concatenate the DataFrames side by side
final_df = pd.concat(aligned_dataframes, axis=1)

# Save the combined DataFrame to a new CSV file
#**存取路徑
final_df.to_csv('/content/drive/MyDrive/Machine-learning-project/Xception/1123_result/F/F_prediction.csv', index=False)

print("Combined CSV file has been created as 'combined_groups.csv'")

In [ ]:
##把validation出來的資料集中在一資料當中
#有**符號的是要更改的部分

#**檔案路徑
train_csv ='/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/training_history.csv'
M_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/val_M.csv'
F_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/val_F.csv'
young_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/val_young.csv'
middle_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/val_middle.csv'
old_csv = '/content/drive/MyDrive/Machine-learning-project/Xception/result/10_Epochs_F_1120_0143/val_old.csv'

# File paths for the 6 input files
file_paths = [train_csv,M_csv, F_csv, young_csv, middle_csv,old_csv]

# Group names corresponding to each file
group_names = ['train','M', 'F', 'young', 'middle','old']

# List to hold processed DataFrames
processed_data = []

# Process each file
for file_path, group_name in zip(file_paths, group_names):
    # Read the CSV file
    df = pd.read_csv(file_path, header=None)

    # Extract the first 15 rows and the first 6 columns
    df = df.iloc[:15, :6]

    # Check if the first row is not blank
    if not df.iloc[0].isnull().all():
        # Update the first row to include group name in the header
        df.iloc[0] = [f'{group_name}_Topic'] + list(df.iloc[0, 1:])

    # Add a new column to label the group
    df.insert(0, 'Group', group_name)

    # Append the processed DataFrame to the list
    processed_data.append(df)
    blank_row = pd.DataFrame([[''] * len(df.columns)], columns=df.columns)
    processed_data.append(blank_row)

# Combine all processed DataFrames into a single DataFrame
combined_df = pd.concat(processed_data, ignore_index=True)

# Save the combined DataFrame to a new CSV file
#**存取路徑
combined_df.to_csv('/content/drive/MyDrive/Machine-learning-project/Xception/1123_result/F/F_val.csv', index=False, header=False)

print("Combined CSV file has been created as 'combined_extracted_data.csv'")


In [ ]:
##畫roc圖
#有**符號的是要更改的部分

#**檔案路徑
file_csv ='/content/drive/MyDrive/Machine-learning-project/Xception/1123_result/F/F_roc_F_separated_classes.csv'

import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file, ignoring the first row (header)
df = pd.read_csv(file_csv, header=1)  # Skip the first row

# Extracting x and y pairs (x1, y1), (x2, y2), (x3, y3), (x4, y4), (x5, y5)
x1, y1 = df.iloc[:, 0], df.iloc[:, 1]  # x1, y1
x2, y2 = df.iloc[:, 4], df.iloc[:, 5]  # x2, y2
x3, y3 = df.iloc[:, 8], df.iloc[:, 9]  # x3, y3
x4, y4 = df.iloc[:, 12], df.iloc[:, 13] # x4, y4
#x5, y5 = df.iloc[:, 16], df.iloc[:, 17] # x5, y5

# Plot each of the five lines with different colors
plt.figure(figsize=(6, 6))
plt.plot(x1, y1, label='Nodule')#, linestyle='-',color='b')
plt.plot(x2, y2, label='Infiltration')#, linestyle='-',color='g')
plt.plot(x3, y3, label='Effusion')#, linestyle='-',color='r')
plt.plot(x4, y4, label='Atelectasis')#, linestyle='-',color='c')
#plt.plot(x5, y5, label='all')#, linestyle='-',color='m')

# Add labels and title
plt.xlabel('FPR')
plt.ylabel('TPR')
#**圖名稱
plt.title('F_ROC')

# Show the legend
plt.legend()

#**存取路徑
# Save the plot as a PNG file
plt.savefig('/content/drive/MyDrive/Machine-learning-project/Xception/1123_result/F/F_roc_F_plot.png', format='png')

# Show the plot
plt.show()